In [1]:
#using MPI
#MPI.Init()
include("pulsed_squeezed_fiber_lasers.jl")

prop_system (generic function with 1 method)

In [2]:
λ_center = 1560e-9;
ω_center = 2*pi*c_light / λ_center;
fiber_index = 1.47;
v_group = c_light / fiber_index;
fiber_dispersion = 1*22e3 * 1e-30 * v_group^3; # in m^2/s
γ_fiber = 1.8*1e-3; # in 1/W/m.
fiber_nonlinearity = ħ*(ω_center)*(v_group^2)*(γ_fiber);
L_fiber = 10; # in meters
T_fiber = L_fiber / v_group;

# Pulse parameters - pulse time only makes sense for single-parameter pulses like sech / gaussian
t_pulse = 100e-15; # in seconds
L_pulse = v_group * t_pulse;

L_sim = 50*L_pulse;
N_z = 2^8;
z_grid = collect(range(-L_sim/2,L_sim/2,length=N_z));

N_t = 200;
dt = T_fiber / N_t;
t_grid = collect(range(0,T_fiber,length=N_t));



In [10]:

sim_fbs = sim(2,2^8,z_grid);

power_list = [0.01:0.8:2.5;]*1e8;
println(length(power_list))
t_list = [0.01:0.2:0.3;]
println(length(t_list))


#comm = MPI.COMM_WORLD
rank = 0 #MPI.Comm_rank(MPI.COMM_WORLD)
nprocs = 2 #MPI.Comm_size(MPI.COMM_WORLD)


#ncores = parse(Int, ARGS[2])
#core_i = parse(Int, ARGS[1])

split_power_lists = approx_split(power_list, nprocs)
list_lengths = Array{Int64}(undef, nprocs)

for i in 1:nprocs
    	list_lengths[i] = length(split_power_lists[i])
end

my_power_list = split_power_lists[rank+1]

println(length(power_list))

myphotons_in = zeros(length(my_power_list), length(t_list));
myphotons_out = zeros(length(my_power_list), length(t_list), sim_fbs.num_modes);
myphotons_fluc = 1.0im*zeros(length(my_power_list), length(t_list), sim_fbs.num_modes);

V_vac = vacuum_V(sim_fbs);
#=
bs1 = beamsplitter([1,2],sqrt(0.1));
fiber1 = fiber(1,fiber_index,L_fiber,fiber_dispersion,fiber_nonlinearity);
fiber2 = fiber(2,fiber_index,L_fiber,fiber_dispersion,fiber_nonlinearity);
bs2 = beamsplitter([1,2],sqrt(0.1));
components = [bs1 fiber1 fiber2 bs2];
=#
for ii = 1:length(my_power_list)
    for jj = 1:length(t_list)
        
        bs1 = beamsplitter([1,2],sqrt(t_list[jj]));
        fiber1 = fiber(1,fiber_index,L_fiber,fiber_dispersion,fiber_nonlinearity);
        fiber2 = fiber(2,fiber_index,L_fiber,fiber_dispersion,fiber_nonlinearity);
        bs2 = beamsplitter([1,2],sqrt(t_list[jj]));
        components = [bs1 fiber1 fiber2 bs2];
        
        # Initializing the mean fields of the initial state
        println("flag 1")
        center_amplitude = sqrt(my_power_list[ii]);
        state_sagnac = state(1.0im*zeros((sim_fbs.num_modes)*(sim_fbs.N_z)),V_vac);
        range_mode1 = get_row_index(1,fiber1.fiber_mode):get_row_index(N_z,fiber1.fiber_mode);
        state_sagnac.mean_fields[range_mode1] .= center_amplitude * sech.(sim_fbs.z_grid/L_pulse);
        myphotons_in[ii] = sum(abs2.(get_meanfield_i(state_sagnac,1)));

        # Solving for mean-field and fluctuation dynamics
        println("flag 2")
        prop_system(components,state_sagnac,sim_fbs,t_grid)

        for ss=1:sim_fbs.num_modes
            myphotons_out[ii,jj,ss] = sum(abs2.(get_meanfield_i(state_sagnac,ss)));
            myphotons_fluc[ii,jj,ss] = n2_exp(state_sagnac,ss,sim_fbs);
        end
        #println("(thread $(Threads.threadid()) of $(Threads.nthreads()))")
        #println(ii)
    end
end

mydata = (myphotons_in, myphotons_out, myphotons_fluc)
println(typeof(mydata))
println(mydata)
if rank > 0
	println("$rank: Sending mydata $rank -> 0/n")
	#MPI.send(mydata, 0, rank+nprocs, comm)

else # rank == 0
    
   	#Receive counts from each rank
    	alldata  = [(Matrix{Float64}(undef, 0,0), Array{Float64}(undef, 0,0,0), Array{ComplexF64}(undef, 0,0,0)) for _ in 1:nprocs]
    	photons_in = zeros(0,length(t_list));
    	photons_out = zeros(0,length(t_list),sim_fbs.num_modes);
    	photons_fluc = 1.0im*zeros(0,length(t_list),sim_fbs.num_modes);
        println(typeof(alldata))
    
    	alldata[1] = mydata
	println(mydata)
    	#photons_in[1:length(myphotons_in)] = myphotons_in
    	for i in 1:nprocs-1
        	println("Receiving from $i")
            alldata[i+1] = mydata #,statrcv = MPI.recv(i, i+nprocs, comm)
    	end

    	for i = 1:nprocs
            data = alldata[i]
        	global photons_in = vcat(photons_in, data[1])
		println("in done")
        	global photons_out = vcat(photons_out, data[2])	
		println("out done")
        	global photons_fluc = vcat(photons_fluc, data[3])
		println("fluc done")
    	end
    
    println(photons_in)
    println(photons_out)
    println(photons_fluc)

	if !isdir("outputs")
		mkdir("outputs")
	end

	#writedlm( "outputs/Test_photons_in.csv",  photons_in, ',')
	#writedlm( "outputs/Test_photons_out.csv",  photons_out, ',')
	#writedlm( "outputs/Test_photons_fluc.csv",  photons_fluc, ',')

end
println("Done!")
#MPI.Finalize()


4
2
4
flag 1
flag 2
flag 1
flag 2
flag 1
flag 2
flag 1
flag 2
Tuple{Matrix{Float64}, Array{Float64, 3}, Array{ComplexF64, 3}}
([1.02e7 0.0; 8.262e8 0.0], [9.796112822736705e6 3.4314726626432296e6; 8.136851162032589e8 4.253080590565034e8]

[403887.17726351 6.768527337356978e6; 1.2514883796755534e7 4.0089194094351244e8], ComplexF64[9.79173164836493e6 - 2.9103830456733704e-11im 3.4311477170588938e6 + 6.548361852765083e-11im; 8.670684108812017e8 + 0.0im 1.3361913790157704e9 - 1.1920928955078125e-7im]

ComplexF64[404839.792813089 + 7.275957614183426e-12im 6.7765767627616795e6 + 0.0im; 1.2849097295313448e8 + 7.450580596923828e-9im 3.767295093144072e9 + 0.0im])
Vector{Tuple{Matrix{Float64}, Array{Float64, 3}, Array{ComplexF64, 3}}}
([1.02e7 0.0; 8.262e8 0.0], [9.796112822736705e6 3.4314726626432296e6; 8.136851162032589e8 4.253080590565034e8]

[403887.17726351 6.768527337356978e6; 1.2514883796755534e7 4.0089194094351244e8], ComplexF64[9.79173164836493e6 - 2.9103830456733704e-11im 3.43114771705

In [11]:
squeezing_sing = 10log10.(photons_fluc ./ photons_out);

In [17]:
using Plots
# Converts elements of power_list and T-list to indices of squeezing_sing array
function f(x_var,y_var)
    x_diff =power_list[2] - power_list[1]
    y_diff = t_list[2] - t_list[1]
    x_i = round(Int, (x_var - power_list[1])/x_diff) + 1
    y_i = round(Int, (y_var - t_list[1])/y_diff) + 1
    #println(x_i)
    #println(y_i)
    return real(squeezing_sing[x_i,y_i,1])
end

x = power_list
y = t_list

Plots.contour(x, y, f, fill=true)#, c=cgrad(:hot))
Plots.xlabel!("Input Pulse Energy")
Plots.ylabel!("Splitting Coefficient")

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1317


LoadError: InitError: could not load library "/state/partition1/llgrid/pkg/julia/julia-1.6.1/local/share/julia/artifacts/e086922a5a3c20ca3e6866a33e42d8ec4689553e/lib/libgio-2.0.so"
/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/libgobject-2.0.so.0: undefined symbol: g_memdup2
during initialization of module Glib_jll